let's download all the dependencies that we will need.

In [3]:
import numpy as np
import pandas as pd
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans


Solving environment: ...working... done

# All requested packages already installed.



Tranform the data into a pandas dataframe

In [12]:
pclg_data = pd.read_csv('pclg.csv')

In [14]:
pclg_data.head()

,Unnamed: 0,Postcode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [16]:
pclg_data.drop(['Unnamed: 0'],axis = 1)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


In [37]:
latitude = 43.7
longitude = -79.4

Create a map of Toronto with neighborhoods superimposed on top.

In [48]:
map_pclg = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(pclg_data['Latitude'], pclg_data['Longitude'], pclg_data['Borough'], pclg_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_pclg)  
    


In [49]:
map_pclg

In [46]:
toronto_data = pclg_data[pclg_data['Borough'].str.contains('Toronto')]
toronto_data.drop(['Unnamed: 0'],axis = 1)

,Postcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
36,M4E,East Toronto,The Beaches,43.676357,-79.293031
37,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
41,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
42,M6G,Downtown Toronto,Christie,43.669542,-79.422564
49,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568


In [47]:
toronto_data.head()

,Unnamed: 0,Postcode,Borough,Neighborhood,Latitude,Longitude
2,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
13,13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
14,14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
27,27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [58]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    

In [59]:
map_toronto